# Capstone - Push data Chunk
### 1st Survey **(Student Experience Survey)**

## Importing & Authenticating

In [ ]:
#Authenticating google colab
import google.colab
google.colab.auth.authenticate_user()

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('omw-1.4')
nltk.download('wordnet')
nltk.download('vader_lexicon')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [ ]:
!pip install pycountry
!pip install fuzzywuzzy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Importing Libraries
import warnings
warnings.filterwarnings("ignore")
import gspread
import io
import pandas as pd
import numpy as np
import pycountry
import re
pd.set_option('display.max_columns', None)
import pandas_gbq

from google.oauth2 import service_account
from fuzzywuzzy import fuzz
from google.cloud import bigquery
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer 
from nltk.stem import WordNetLemmatizer
from nltk.sentiment import SentimentIntensityAnalyzer
from textblob import TextBlob


In [ ]:
# authenticate with Google using the service account credentials
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
creds = service_account.Credentials.from_service_account_file('/content/drive/MyDrive/.cred/cred.json', scopes=scope)
client = gspread.authorize(creds)

# open the sheet by its title
sheet = client.open('Student_Survey_1').sheet1

# get all values from the sheet
values = sheet.get_all_values()

# convert values to a Pandas dataframe
df = pd.DataFrame(values[1:], columns=values[0])

In [ ]:
dd= df.copy()

In [ ]:
df.head(3)

,Timestamp,In which cohort (Intake) did you start the DAB program?,Which semester are you currently enrolled in?,What is your country of origin?,Could you provide the state/province of origin for the above-mentioned country?,Which of the following age group best describes you?,Which gender do you identify yourself as?,What was the highest level of education you completed (prior to starting the DAB program)?,What was your major field of study as per your highest degree or level of education?,Do you have any prior work experience?,How many years of experience do you have?,In which sector/industry have you had previous work experience?,Please provide zip code for your current accommodation in Windsor-Essex County region.,What type of accommodation do you have?,"Evaluate your living experience in regards to your housing, using the below-mentioned scale [Ease of finding accommodation]","Evaluate your living experience in regards to your housing, using the below-mentioned scale [Quality of accommodation]","Evaluate your living experience in regards to your housing, using the below-mentioned scale [Affordability of Living]","Evaluate your living experience in regards to your housing, using the below-mentioned scale [Reliability of commute]","Evaluate your living experience in regards to your housing, using the below-mentioned scale [Availability of daily needs]","Where did you find information regarding facilities like accommodation, transport, food, and services in Windsor-Essex ?",What is your current employment status?,"On a scale of 1 to 5, rate the options below in terms of your academic experience. [Timetable Schedules]","On a scale of 1 to 5, rate the options below in terms of your academic experience. [Section Allocation (every semester)]","On a scale of 1 to 5, rate the options below in terms of your academic experience. [Weekly Class Hours]","On a scale of 1 to 5, rate the options below in terms of your academic experience. [Availability of Course Contents, Reference Guides, Study Environments and Labs]","On a scale of 1 to 5, rate the options below in terms of your academic experience. [Academic Workload after college Hours (Assignment, Labs...)]",Are you aware or use any of the student services offered by the college/SRC?,Which services do you use?,"Please provide any additional feedback, appreciation, or suggestions regarding your experience."
0,,Fall 2021,4th Semester,India,Delhi,36-45,Male,"Master's degree (MCA, MBA, M.Tech, MSc...)",Business/Administration,Yes,10 - 15 Years,Banking and Financial services,N9A 5B4,Private Room,Dissatisfied,Satisfied,Dissatisfied,Satisfied,Satisfied,,Unemployed,3,1,2,3,3,"No, I am not aware.",Other;,There's lack of dissemination of information t...
1,,Winter 2022,4th Semester,India,Karnataka,26-35,Male,"Master's degree (MCA, MBA, M.Tech, MSc...)",Technology (Computer Science/Information Techn...,Yes,6 - 10 Years,Information Technology,N9A5B5,Private Room,Satisfied,Satisfied,Satisfied,Satisfied,Satisfied,,Part-time,3,Satisfied,Satisfied,Satisfied,3,"Yes, I am aware but don't use.",Health and Wellness ;,A cafeteria in the downtown campus is much needed
2,,Winter 2022,4th Semester,India,Punjab,36-45,Male,"Bachelor's degree (BSc, B.Ed., B.Tech, B.Law...)",Technology (Computer Science/Information Techn...,Yes,15 + Years,Information Technology,N9A 4B8,Private Room,Satisfied,Dissatisfied,Satisfied,Satisfied,Satisfied,,Part-time,4,3,2,4,3,"Yes, I am aware and use.",Tutoring services;Sports Scholarship;,awareness regarding college services should be...


## Creating a DataFrame




In [ ]:
#Creating a DataFrame Object - df and renaming it columns to a particular format , datatypes are corrected and the unwanted columns are dropped

df = df.rename(columns={
    'In which cohort (Intake) did you start the DAB program?': 'Intake',
    'Which semester are you currently enrolled in?': 'Semester',
    'What is your country of origin?': 'Country',
    'Could you provide the state/province of origin for the above-mentioned country?': 'State',
    'Which of the following age group best describes you?': 'Age',
    'Which gender do you identify yourself as?': 'Gender',
    'What was the highest level of education you completed (prior to starting the DAB program)?': 'Education_Lvl',
    'What was your major field of study as per your highest degree or level of education?': 'Majors',
    'Do you have any prior work experience?': 'Work_Exp',
    'How many years of experience do you have?': 'Yrs_Exp',
    'In which sector/industry have you had previous work experience?': 'Field_of_Exp',
    'Please provide zip code for your current accommodation in Windsor-Essex County region.': 'Zip_code',
    'What type of accommodation do you have?': 'Accm_Type',
    'Evaluate your living experience in regards to your housing, using the below-mentioned scale [Ease of finding  accommodation]': 'Accm_finding_score',
    'Evaluate your living experience in regards to your housing, using the below-mentioned scale [Quality of accommodation]': 'Accm_Quality_Score',
    'Evaluate your living experience in regards to your housing, using the below-mentioned scale [Affordability of Living]': 'Accm_affordibility_score',
    'Evaluate your living experience in regards to your housing, using the below-mentioned scale [Reliability of commute]': 'Commute_score',
    'Evaluate your living experience in regards to your housing, using the below-mentioned scale [Availability of daily needs]': 'Needs_Availibility_Score',
    'Where did you find information regarding facilities like accommodation, transport, food, and services in Windsor-Essex ? ': 'Accm_info_source',
    ' What is your current employment status?' : 'employment_status',
    'On a scale of 1 to 5, rate the options below in terms of your academic experience. [Timetable Schedules]': 'Timetable_Schedules_score',
    'On a scale of 1 to 5, rate the options below in terms of your academic experience. [Section Allocation (every semester)]': 'Section_Allocation_score',
    'On a scale of 1 to 5, rate the options below in terms of your academic experience. [Weekly Class Hours]': 'Class_Hours_Score',
    'On a scale of 1 to 5, rate the options below in terms of your academic experience. [Availability of Course Contents, Reference Guides, Study Environments and Labs]': 'Course_Contents_Score',
    'On a scale of 1 to 5, rate the options below in terms of your academic experience. [Academic Workload after college Hours (Assignment, Labs...)]': 'Afterclass_Workload_score',
    'Are you aware or use any of the student services offered by the college/SRC?': 'Services_Awareness',
    'Which services do you use?': 'Services_Used',
    'Please provide any additional feedback, appreciation, or suggestions regarding your experience.': 'Feedback'
})

drop = ['Timestamp']

for col in drop:
    if col in df.columns:
        df = df.drop(columns=[col])
df.insert(0, 'ID', range(1, len(df) + 1))
df.head(3)

,ID,Intake,Semester,Country,State,Age,Gender,Education_Lvl,Majors,Work_Exp,Yrs_Exp,Field_of_Exp,Zip_code,Accm_Type,Accm_finding_score,Accm_Quality_Score,Accm_affordibility_score,Commute_score,Needs_Availibility_Score,Accm_info_source,employment_status,Timetable_Schedules_score,Section_Allocation_score,Class_Hours_Score,Course_Contents_Score,Afterclass_Workload_score,Services_Awareness,Services_Used,Feedback
0,1,Fall 2021,4th Semester,India,Delhi,36-45,Male,"Master's degree (MCA, MBA, M.Tech, MSc...)",Business/Administration,Yes,10 - 15 Years,Banking and Financial services,N9A 5B4,Private Room,Dissatisfied,Satisfied,Dissatisfied,Satisfied,Satisfied,,Unemployed,3,1,2,3,3,"No, I am not aware.",Other;,There's lack of dissemination of information t...
1,2,Winter 2022,4th Semester,India,Karnataka,26-35,Male,"Master's degree (MCA, MBA, M.Tech, MSc...)",Technology (Computer Science/Information Techn...,Yes,6 - 10 Years,Information Technology,N9A5B5,Private Room,Satisfied,Satisfied,Satisfied,Satisfied,Satisfied,,Part-time,3,Satisfied,Satisfied,Satisfied,3,"Yes, I am aware but don't use.",Health and Wellness ;,A cafeteria in the downtown campus is much needed
2,3,Winter 2022,4th Semester,India,Punjab,36-45,Male,"Bachelor's degree (BSc, B.Ed., B.Tech, B.Law...)",Technology (Computer Science/Information Techn...,Yes,15 + Years,Information Technology,N9A 4B8,Private Room,Satisfied,Dissatisfied,Satisfied,Satisfied,Satisfied,,Part-time,4,3,2,4,3,"Yes, I am aware and use.",Tutoring services;Sports Scholarship;,awareness regarding college services should be...


In [ ]:
len(df)

353

## Data Cleaning

In [ ]:
#State (Text Field in Sruvey) - Using Fuzzywuzzy, validating the entered state with a valid state and also formatting it to desirable format
def is_valid_state(state):
    state_upper = state.upper()
    for subdivision in pycountry.subdivisions:
        subdivision_upper = subdivision.name.upper()
        score = fuzz.ratio(state_upper, subdivision_upper)
        if score >= 80: 
            return subdivision_upper
    return float('nan')
df['State'] = df['State'].apply(lambda x: is_valid_state(x))
df.head(3)

,ID,Intake,Semester,Country,State,Age,Gender,Education_Lvl,Majors,Work_Exp,Yrs_Exp,Field_of_Exp,Zip_code,Accm_Type,Accm_finding_score,Accm_Quality_Score,Accm_affordibility_score,Commute_score,Needs_Availibility_Score,Accm_info_source,employment_status,Timetable_Schedules_score,Section_Allocation_score,Class_Hours_Score,Course_Contents_Score,Afterclass_Workload_score,Services_Awareness,Services_Used,Feedback
0,1,Fall 2021,4th Semester,India,DELHI,36-45,Male,"Master's degree (MCA, MBA, M.Tech, MSc...)",Business/Administration,Yes,10 - 15 Years,Banking and Financial services,N9A 5B4,Private Room,Dissatisfied,Satisfied,Dissatisfied,Satisfied,Satisfied,,Unemployed,3,1,2,3,3,"No, I am not aware.",Other;,There's lack of dissemination of information t...
1,2,Winter 2022,4th Semester,India,KARNĀTAKA,26-35,Male,"Master's degree (MCA, MBA, M.Tech, MSc...)",Technology (Computer Science/Information Techn...,Yes,6 - 10 Years,Information Technology,N9A5B5,Private Room,Satisfied,Satisfied,Satisfied,Satisfied,Satisfied,,Part-time,3,Satisfied,Satisfied,Satisfied,3,"Yes, I am aware but don't use.",Health and Wellness ;,A cafeteria in the downtown campus is much needed
2,3,Winter 2022,4th Semester,India,PUNJAB,36-45,Male,"Bachelor's degree (BSc, B.Ed., B.Tech, B.Law...)",Technology (Computer Science/Information Techn...,Yes,15 + Years,Information Technology,N9A 4B8,Private Room,Satisfied,Dissatisfied,Satisfied,Satisfied,Satisfied,,Part-time,4,3,2,4,3,"Yes, I am aware and use.",Tutoring services;Sports Scholarship;,awareness regarding college services should be...


In [ ]:
#Field of Experience (Text Field in Survey) - Filling the missing values with No Experience and that grouping it and storing it in a new column

df['Field_of_Exp'] = df['Field_of_Exp'].fillna('No Experience')

def group_titles(title):
    if 'No Experience' in title:
        return 'No Experience'
    elif 'IT' in title or 'Information Technology' in title or 'Software' in title or 'it' in title:
        return 'Information Technology'
    elif 'Banking' in title or 'Finance' in title or 'Accounting' in title or 'FinTech' in title:
        return 'Finance & Banking'
    elif 'Data' in title or 'Analytics' in title or 'Statistician' in title or 'data' in title or 'Business Data Analyst' in title:
        return 'Data Analytics'
    elif 'Marketing' in title or 'Public Relations' in title or 'Business Development' in title or 'Sales' in title or 'Account Manager' in title or 'Digital Marketing' in title or 'Content Marketing' in title or 'Marketing Analytics' in title:
        return 'Sales & Marketing'
    elif 'Manufacturing' in title or 'Operations' in title or 'Supply Chain' in title:
        return 'Manufacturing'
    elif 'Healthcare' in title or 'Medical' in title or 'Pharmaceutical' in title or 'Health Insurance' in title:
        return 'Healthcare'
    elif 'Media' in title or 'Entertainment' in title or 'Broadcasting' in title:
        return 'Media/Entertainment'
    elif 'Education' in title or 'Teaching' in title or 'Research' in title:
        return 'Education'
    elif 'Engineering' in title or 'Mechanical' in title or 'Aeronautical' in title or 'Architecture' in title or 'Electronics' in title or 'Electrical' in title or 'Communication' in title or 'Telecom' in title or 'Automotive' in title:
        return 'Engineering'
    elif 'Customer Service' in title or 'Hospitality' in title or 'Client Services' in title or 'Retail' in title or 'Merchandising' in title:
        return 'Customer Service'
    elif 'Project Management' in title or 'Operations Management' in title or 'Program Management' in title:
        return 'Project Management'
    elif 'Human Resources' in title or 'Recruiting' in title:
        return 'Human Resources'
    elif 'Insurance' in title:
        return 'Insurance'
    elif 'Government' in title or 'Military' in title:
        return 'Government'
    elif 'BPO' in title or 'KPO' in title or 'RPO' in title:
        return 'BPO/KPO/RPO'
    else:
        return 'Other'

        
df['Field_of_Exp'] = df['Field_of_Exp'].astype(str)
df['Field_of_Exp'] = df['Field_of_Exp'].apply(group_titles)
df.head(3)

,ID,Intake,Semester,Country,State,Age,Gender,Education_Lvl,Majors,Work_Exp,Yrs_Exp,Field_of_Exp,Zip_code,Accm_Type,Accm_finding_score,Accm_Quality_Score,Accm_affordibility_score,Commute_score,Needs_Availibility_Score,Accm_info_source,employment_status,Timetable_Schedules_score,Section_Allocation_score,Class_Hours_Score,Course_Contents_Score,Afterclass_Workload_score,Services_Awareness,Services_Used,Feedback
0,1,Fall 2021,4th Semester,India,DELHI,36-45,Male,"Master's degree (MCA, MBA, M.Tech, MSc...)",Business/Administration,Yes,10 - 15 Years,Finance & Banking,N9A 5B4,Private Room,Dissatisfied,Satisfied,Dissatisfied,Satisfied,Satisfied,,Unemployed,3,1,2,3,3,"No, I am not aware.",Other;,There's lack of dissemination of information t...
1,2,Winter 2022,4th Semester,India,KARNĀTAKA,26-35,Male,"Master's degree (MCA, MBA, M.Tech, MSc...)",Technology (Computer Science/Information Techn...,Yes,6 - 10 Years,Information Technology,N9A5B5,Private Room,Satisfied,Satisfied,Satisfied,Satisfied,Satisfied,,Part-time,3,Satisfied,Satisfied,Satisfied,3,"Yes, I am aware but don't use.",Health and Wellness ;,A cafeteria in the downtown campus is much needed
2,3,Winter 2022,4th Semester,India,PUNJAB,36-45,Male,"Bachelor's degree (BSc, B.Ed., B.Tech, B.Law...)",Technology (Computer Science/Information Techn...,Yes,15 + Years,Information Technology,N9A 4B8,Private Room,Satisfied,Dissatisfied,Satisfied,Satisfied,Satisfied,,Part-time,4,3,2,4,3,"Yes, I am aware and use.",Tutoring services;Sports Scholarship;,awareness regarding college services should be...


In [ ]:
#Zipcode - Formatting Zipcode in a desirable format

pattern = re.compile('[A-Z0-9]')

def format_zip(zip_code):
    if isinstance(zip_code, str) and re.match('\s*[A-Z0-9]{6}\s*', zip_code):
        alphanumeric_chars = pattern.findall(zip_code)
        padded_string = ''.join(alphanumeric_chars)[:6].ljust(6)
        formatted_string = padded_string[:3] + ' ' + padded_string[3:]
        return formatted_string
    else:
        return zip_code

df['Zip_code'] = df['Zip_code'].apply(format_zip)
df.head(3)

,ID,Intake,Semester,Country,State,Age,Gender,Education_Lvl,Majors,Work_Exp,Yrs_Exp,Field_of_Exp,Zip_code,Accm_Type,Accm_finding_score,Accm_Quality_Score,Accm_affordibility_score,Commute_score,Needs_Availibility_Score,Accm_info_source,employment_status,Timetable_Schedules_score,Section_Allocation_score,Class_Hours_Score,Course_Contents_Score,Afterclass_Workload_score,Services_Awareness,Services_Used,Feedback
0,1,Fall 2021,4th Semester,India,DELHI,36-45,Male,"Master's degree (MCA, MBA, M.Tech, MSc...)",Business/Administration,Yes,10 - 15 Years,Finance & Banking,N9A 5B4,Private Room,Dissatisfied,Satisfied,Dissatisfied,Satisfied,Satisfied,,Unemployed,3,1,2,3,3,"No, I am not aware.",Other;,There's lack of dissemination of information t...
1,2,Winter 2022,4th Semester,India,KARNĀTAKA,26-35,Male,"Master's degree (MCA, MBA, M.Tech, MSc...)",Technology (Computer Science/Information Techn...,Yes,6 - 10 Years,Information Technology,N9A 5B5,Private Room,Satisfied,Satisfied,Satisfied,Satisfied,Satisfied,,Part-time,3,Satisfied,Satisfied,Satisfied,3,"Yes, I am aware but don't use.",Health and Wellness ;,A cafeteria in the downtown campus is much needed
2,3,Winter 2022,4th Semester,India,PUNJAB,36-45,Male,"Bachelor's degree (BSc, B.Ed., B.Tech, B.Law...)",Technology (Computer Science/Information Techn...,Yes,15 + Years,Information Technology,N9A 4B8,Private Room,Satisfied,Dissatisfied,Satisfied,Satisfied,Satisfied,,Part-time,4,3,2,4,3,"Yes, I am aware and use.",Tutoring services;Sports Scholarship;,awareness regarding college services should be...


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 353 entries, 0 to 352
Data columns (total 29 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   ID                         353 non-null    int64 
 1   Intake                     353 non-null    object
 2   Semester                   353 non-null    object
 3   Country                    353 non-null    object
 4   State                      294 non-null    object
 5   Age                        353 non-null    object
 6   Gender                     353 non-null    object
 7   Education_Lvl              353 non-null    object
 8   Majors                     353 non-null    object
 9   Work_Exp                   353 non-null    object
 10  Yrs_Exp                    353 non-null    object
 11  Field_of_Exp               353 non-null    object
 12  Zip_code                   353 non-null    object
 13  Accm_Type                  353 non-null    object
 14  Accm_findi

In [ ]:
l = ['Timetable_Schedules_score', 'Section_Allocation_score', 'Class_Hours_Score', 'Course_Contents_Score', 'Afterclass_Workload_score']

for col in l:
  df[col] = df[col].replace('Satisfied', 4)


In [ ]:
l = ['Timetable_Schedules_score', 'Section_Allocation_score', 'Class_Hours_Score', 'Course_Contents_Score', 'Afterclass_Workload_score']

for col in l:
    print(df[col].unique())

['3' '4' '2' '1' '5']
['1' 4 '3' '4' '5' '2']
['2' 4 '4' '3' '1' '5']
['3' 4 '4' '5' '2' '1']
['3' '2' '4' '5' '1']


In [ ]:
# Filling all the missing values in below columns with 'None'

df['Services_Used']=df['Services_Used'].replace(np.nan,'None')
df['Feedback']=df['Feedback'].replace(np.nan,'None')
df['Accm_info_source']=df['Accm_info_source'].replace(np.nan,'None')
df['Yrs_Exp']=df['Yrs_Exp'].replace(np.nan,'None')
df['State']=df['State'].replace(np.nan,'None')
df.head(3)

,ID,Intake,Semester,Country,State,Age,Gender,Education_Lvl,Majors,Work_Exp,Yrs_Exp,Field_of_Exp,Zip_code,Accm_Type,Accm_finding_score,Accm_Quality_Score,Accm_affordibility_score,Commute_score,Needs_Availibility_Score,Accm_info_source,employment_status,Timetable_Schedules_score,Section_Allocation_score,Class_Hours_Score,Course_Contents_Score,Afterclass_Workload_score,Services_Awareness,Services_Used,Feedback
0,1,Fall 2021,4th Semester,India,DELHI,36-45,Male,"Master's degree (MCA, MBA, M.Tech, MSc...)",Business/Administration,Yes,10 - 15 Years,Finance & Banking,N9A 5B4,Private Room,Dissatisfied,Satisfied,Dissatisfied,Satisfied,Satisfied,,Unemployed,3,1,2,3,3,"No, I am not aware.",Other;,There's lack of dissemination of information t...
1,2,Winter 2022,4th Semester,India,KARNĀTAKA,26-35,Male,"Master's degree (MCA, MBA, M.Tech, MSc...)",Technology (Computer Science/Information Techn...,Yes,6 - 10 Years,Information Technology,N9A 5B5,Private Room,Satisfied,Satisfied,Satisfied,Satisfied,Satisfied,,Part-time,3,4,4,4,3,"Yes, I am aware but don't use.",Health and Wellness ;,A cafeteria in the downtown campus is much needed
2,3,Winter 2022,4th Semester,India,PUNJAB,36-45,Male,"Bachelor's degree (BSc, B.Ed., B.Tech, B.Law...)",Technology (Computer Science/Information Techn...,Yes,15 + Years,Information Technology,N9A 4B8,Private Room,Satisfied,Dissatisfied,Satisfied,Satisfied,Satisfied,,Part-time,4,3,2,4,3,"Yes, I am aware and use.",Tutoring services;Sports Scholarship;,awareness regarding college services should be...


In [ ]:
convert = ['ID', 'Timetable_Schedules_score', 'Section_Allocation_score', 'Class_Hours_Score', 'Course_Contents_Score', 'Afterclass_Workload_score']
df[convert] = df[convert].astype(int)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 353 entries, 0 to 352
Data columns (total 29 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   ID                         353 non-null    int64 
 1   Intake                     353 non-null    object
 2   Semester                   353 non-null    object
 3   Country                    353 non-null    object
 4   State                      353 non-null    object
 5   Age                        353 non-null    object
 6   Gender                     353 non-null    object
 7   Education_Lvl              353 non-null    object
 8   Majors                     353 non-null    object
 9   Work_Exp                   353 non-null    object
 10  Yrs_Exp                    353 non-null    object
 11  Field_of_Exp               353 non-null    object
 12  Zip_code                   353 non-null    object
 13  Accm_Type                  353 non-null    object
 14  Accm_findi

In [ ]:
df.isnull().sum()

ID                           0
Intake                       0
Semester                     0
Country                      0
State                        0
Age                          0
Gender                       0
Education_Lvl                0
Majors                       0
Work_Exp                     0
Yrs_Exp                      0
Field_of_Exp                 0
Zip_code                     0
Accm_Type                    0
Accm_finding_score           0
Accm_Quality_Score           0
Accm_affordibility_score     0
Commute_score                0
Needs_Availibility_Score     0
Accm_info_source             0
employment_status            0
Timetable_Schedules_score    0
Section_Allocation_score     0
Class_Hours_Score            0
Course_Contents_Score        0
Afterclass_Workload_score    0
Services_Awareness           0
Services_Used                0
Feedback                     0
dtype: int64

## Big-Query Table
### API Connection to Google BigQuery Database (surveyproject-378222.Capstone_Project)

In [ ]:
#Connecting Big-Query Project 
client = bigquery.Client(project='surveyproject-378222')
sc = """
DROP TABLE IF EXISTS `surveyproject-378222.Capstone_Project.Base_Survey_Table`;
"""
query_job = client.query(sc)
query_job.result()
table_id = 'surveyproject-378222.Capstone_Project.Base_Survey_Table'

# Defining the schema for the new table
schema = [
    bigquery.SchemaField('ID', 'INTEGER', mode='REQUIRED'),
    bigquery.SchemaField('Intake', 'STRING'),
    bigquery.SchemaField('Semester', 'STRING'),
    bigquery.SchemaField('Country', 'STRING'),
    bigquery.SchemaField('State', 'STRING'),
    bigquery.SchemaField('Age', 'STRING'),
    bigquery.SchemaField('Gender', 'STRING'),
    bigquery.SchemaField('Education_Lvl', 'STRING'),
    bigquery.SchemaField('Majors', 'STRING'),
    bigquery.SchemaField('Work_Exp', 'STRING'),
    bigquery.SchemaField('Yrs_Exp', 'STRING'),
    bigquery.SchemaField('Field_of_Exp', 'STRING'),
    bigquery.SchemaField('Zip_code', 'STRING'),
    bigquery.SchemaField('Accm_Type', 'STRING'),
    bigquery.SchemaField('Accm_finding_score', 'STRING'),
    bigquery.SchemaField('Accm_Quality_Score', 'STRING'),
    bigquery.SchemaField('Accm_affordibility_score', 'STRING'),
    bigquery.SchemaField('Commute_score', 'STRING'),
    bigquery.SchemaField('Needs_Availibility_Score', 'STRING'),
    bigquery.SchemaField('Accm_info_source', 'STRING'),
    bigquery.SchemaField('employment_status', 'STRING'),
    bigquery.SchemaField('Timetable_Schedules_score', 'INTEGER'),
    bigquery.SchemaField('Section_Allocation_score', 'INTEGER'),
    bigquery.SchemaField('Class_Hours_Score', 'INTEGER'),
    bigquery.SchemaField('Course_Contents_Score', 'INTEGER'),
    bigquery.SchemaField('Afterclass_Workload_score', 'INTEGER'),
    bigquery.SchemaField('Services_Awareness', 'STRING'),
    bigquery.SchemaField('Services_Used', 'STRING'),
    bigquery.SchemaField('Feedback', 'STRING')
]

try:
    # Try to get the existing table
    table = client.get_table(table_id)
    print(f'Table {table.table_id} already exists in BigQuery.')
except google.api_core.exceptions.NotFound:
    # If the table does not exist, create it
    table = bigquery.Table(table_id, schema=schema)
    table = client.create_table(table)
    print(f'Table {table.table_id} created in BigQuery.')


project_id = 'surveyproject-378222'

#Inserting Data into to Big-Query Base Table
client = bigquery.Client(project=project_id)
job_config = bigquery.LoadJobConfig()
job_config.source_format = bigquery.SourceFormat.CSV
job_config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE
job = client.load_table_from_dataframe(df, table_id, job_config=job_config)
job.result()

Table Base_Survey_Table created in BigQuery.


LoadJob<project=surveyproject-378222, location=northamerica-northeast2, id=7388c470-1708-4f5d-8df8-9568488422e5>

## Breaking Base_Survey_Table to 7 other Tables

In [ ]:
 #Defining the SQL code to create tables and insert data
sql_code = """
DROP TABLE IF EXISTS `Capstone_Project.Academic`;
CREATE TABLE `Capstone_Project.Academic` (
  ID INT NOT NULL,
  intake STRING,
  semester STRING
);
INSERT INTO `Capstone_Project.Academic`
SELECT ID, intake, semester 
FROM `surveyproject-378222.Capstone_Project.Base_Survey_Table`;

DROP TABLE IF EXISTS `Capstone_Project.Academic_feedback`;
CREATE TABLE `Capstone_Project.Academic_feedback` (
  ID INT NOT NULL,
  Timetable_Schedules_score INT,
  Section_Allocation_score INT,
  Class_Hours_Score INT,
  Course_Contents_Score INT,
  Afterclass_Workload_score INT
);
INSERT INTO `Capstone_Project.Academic_feedback`
SELECT ID, Timetable_Schedules_score, Section_Allocation_score, Class_Hours_Score, Course_Contents_Score, Afterclass_Workload_score 
FROM `surveyproject-378222.Capstone_Project.Base_Survey_Table`;

DROP TABLE IF EXISTS `Capstone_Project.Accommodation`;
CREATE TABLE `Capstone_Project.Accommodation` (
  ID INT NOT NULL,
  Zip_code STRING,
  Accm_Type STRING,
  Accm_info_source STRING
);
INSERT INTO `Capstone_Project.Accommodation`
SELECT ID, Zip_code, Accm_Type, Accm_info_source
FROM `surveyproject-378222.Capstone_Project.Base_Survey_Table`;

DROP TABLE IF EXISTS `Capstone_Project.Accommodation_feedback`;
CREATE TABLE `Capstone_Project.Accommodation_feedback` 
(
  ID INT NOT NULL,
  Accm_finding_score STRING,
  Accm_Quality_Score STRING,
  Accm_affordibility_score STRING,
  Commute_score STRING,
  Needs_Availibility_Score STRING
);

 insert into `surveyproject-378222.Capstone_Project.Accommodation_feedback`
select 
  ID,
Accm_finding_score ,
  Accm_Quality_Score ,
  Accm_affordibility_score ,
  Commute_score ,
  Needs_Availibility_Score 
from `surveyproject-378222.Capstone_Project.Base_Survey_Table` ;

DROP TABLE IF EXISTS `Capstone_Project.Demographics`;
CREATE TABLE `Capstone_Project.Demographics` 
(
  ID INT NOT NULL,
  Country STRING,
  State STRING ,
  Age STRING,
  Gender STRING,
  Education_lvl STRING,
  Majors STRING,
  Work_exp STRING,
  Yrs_exp STRING,
  Field_of_exp STRING,
  employment_status STRING,
);

insert into `surveyproject-378222.Capstone_Project.Demographics`
select 
  ID,
  Country ,
  State ,
  Age ,
  Gender ,
  Education_lvl ,
  Majors ,
  Work_exp ,
  Yrs_exp ,
  Field_of_exp ,
  employment_status
from `surveyproject-378222.Capstone_Project.Base_Survey_Table`;

DROP TABLE IF EXISTS `Capstone_Project.Feedback`;
CREATE TABLE `Capstone_Project.Feedback` (
  ID INT NOT NULL,
  Feedback STRING
);

INSERT INTO `Capstone_Project.Feedback` (ID, Feedback)
SELECT ID, Feedback FROM `Capstone_Project.Base_Survey_Table`;


DROP TABLE IF EXISTS `Capstone_Project.Services`;
CREATE TABLE `Capstone_Project.Services` 
(
  ID INT NOT NULL,
  Services_Awareness STRING,
  Services_Used STRING
);

insert into `surveyproject-378222.Capstone_Project.Services`
select 
  ID,
  Services_Awareness,
  Services_Used
from `surveyproject-378222.Capstone_Project.Base_Survey_Table`;

"""
query_job = client.query(sql_code)
query_job.result()


## Creating Academic and Demographic combine table 

In [ ]:
Demographics= pandas_gbq.read_gbq(f'SELECT * FROM `surveyproject-378222.Capstone_Project.Demographics`',project_id='surveyproject-378222', dialect='standard') 
Academic_feedback = pandas_gbq.read_gbq(f'SELECT * FROM `surveyproject-378222.Capstone_Project.Academic_feedback`',project_id='surveyproject-378222', dialect='standard')

Downloading: 100%|██████████|
Downloading: 100%|██████████|


In [ ]:
AcaDemo = Demographics
maf = pd.melt(Academic_feedback, id_vars=['ID'], var_name='Score_Type', value_name='Score')
AcaDemo= pd.merge(AcaDemo, maf, on='ID')
AcaDemo = AcaDemo.sort_values(by='ID')
AcaDemo.head(3)

,ID,Country,State,Age,Gender,Education_lvl,Majors,Work_exp,Yrs_exp,Field_of_exp,employment_status,Score_Type,Score
967,1,India,DELHI,36-45,Male,"Master's degree (MCA, MBA, M.Tech, MSc...)",Business/Administration,Yes,10 - 15 Years,Finance & Banking,Unemployed,Class_Hours_Score,2
965,1,India,DELHI,36-45,Male,"Master's degree (MCA, MBA, M.Tech, MSc...)",Business/Administration,Yes,10 - 15 Years,Finance & Banking,Unemployed,Timetable_Schedules_score,3
966,1,India,DELHI,36-45,Male,"Master's degree (MCA, MBA, M.Tech, MSc...)",Business/Administration,Yes,10 - 15 Years,Finance & Banking,Unemployed,Section_Allocation_score,1


In [ ]:
#Connecting Big-Query Project 
client = bigquery.Client(project='surveyproject-378222')
sc = """
DROP TABLE IF EXISTS `surveyproject-378222.Capstone_Project.AcaDemo`;
"""
query_job = client.query(sc)
query_job.result()
table_id = 'surveyproject-378222.Capstone_Project.AcaDemo'

# Defining the schema for the new table
schema = [
    bigquery.SchemaField('ID', 'INTEGER', mode='REQUIRED'),
    bigquery.SchemaField('Country', 'STRING'),
    bigquery.SchemaField('State', 'STRING'),
    bigquery.SchemaField('Age', 'STRING'),
    bigquery.SchemaField('Gender', 'STRING'),
    bigquery.SchemaField('Education_Lvl', 'STRING'),
    bigquery.SchemaField('Majors', 'STRING'),
    bigquery.SchemaField('Work_Exp', 'STRING'),
    bigquery.SchemaField('Yrs_Exp', 'STRING'),
    bigquery.SchemaField('Field_of_Exp', 'STRING'),
    bigquery.SchemaField('employment_status', 'STRING'),
    bigquery.SchemaField('Score_Type', 'STRING'),
    bigquery.SchemaField('Score', 'INTEGER'),
  ]

try:
    # Try to get the existing table
    table = client.get_table(table_id)
    print(f'Table {table.table_id} already exists in BigQuery.')
except google.api_core.exceptions.NotFound:
    # If the table does not exist, create it
    table = bigquery.Table(table_id, schema=schema)
    table = client.create_table(table)
    print(f'Table {table.table_id} created in BigQuery.')


project_id = 'surveyproject-378222'

#Inserting Data into to Big-Query Base Table
client = bigquery.Client(project=project_id)
job_config = bigquery.LoadJobConfig()
job_config.source_format = bigquery.SourceFormat.CSV
job_config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE
job = client.load_table_from_dataframe(AcaDemo, table_id, job_config=job_config)
job.result()

Table AcaDemo created in BigQuery.


LoadJob<project=surveyproject-378222, location=northamerica-northeast2, id=2c9a4884-6a49-4947-a7a6-6065ea82158d>

In [ ]:
#Reading data from BigQuery to view the push data
AcaDemo= pandas_gbq.read_gbq(f'SELECT * FROM `surveyproject-378222.Capstone_Project.AcaDemo`',project_id='surveyproject-378222', dialect='standard') 

Downloading: 100%|██████████|


In [ ]:
AcaDemo.head()

,ID,Country,State,Age,Gender,Education_lvl,Majors,Work_exp,Yrs_exp,Field_of_exp,employment_status,Score_Type,Score
0,119,India,TAMIL NĀDU,18-25,Male,"Bachelor's degree (BSc, B.Ed., B.Tech, B.Law...)",Other,No,None,Other,Seeking opportunities currently,Timetable_Schedules_score,3
1,119,India,TAMIL NĀDU,18-25,Male,"Bachelor's degree (BSc, B.Ed., B.Tech, B.Law...)",Other,No,None,Other,Seeking opportunities currently,Section_Allocation_score,3
2,119,India,TAMIL NĀDU,18-25,Male,"Bachelor's degree (BSc, B.Ed., B.Tech, B.Law...)",Other,No,None,Other,Seeking opportunities currently,Class_Hours_Score,3
3,119,India,TAMIL NĀDU,18-25,Male,"Bachelor's degree (BSc, B.Ed., B.Tech, B.Law...)",Other,No,None,Other,Seeking opportunities currently,Course_Contents_Score,4
4,119,India,TAMIL NĀDU,18-25,Male,"Bachelor's degree (BSc, B.Ed., B.Tech, B.Law...)",Other,No,None,Other,Seeking opportunities currently,Afterclass_Workload_score,2


## Creating Academic Score Table 

In [ ]:
#Reading Academic_feedback table from Google BigQuery  
Academic_feedback = pandas_gbq.read_gbq(f'SELECT * FROM `surveyproject-378222.Capstone_Project.Academic_feedback`',project_id='surveyproject-378222', dialect='standard')


Downloading: 100%|██████████|


In [ ]:
af=Academic_feedback.copy()
weights = {'A': 0.25, 'B': 0.25, 'C': 0.0, 'D': 0.25, 'E': 0.25}
af['Acd_score'] = ((af['Timetable_Schedules_score']*weights['A']) + (af['Section_Allocation_score']*weights['B']) + (af['Class_Hours_Score']*weights['C'])+ (af['Course_Contents_Score']*weights['D']) + (af['Afterclass_Workload_score']*weights['E']))
af = af[['ID', 'Acd_score']]
af.head()
Academic_score = af


In [ ]:
#Connecting Big-Query Project 
client = bigquery.Client(project='surveyproject-378222')
sc = """
DROP TABLE IF EXISTS `surveyproject-378222.Capstone_Project.Academic_score`;
"""
query_job = client.query(sc)
query_job.result()
table_id = 'surveyproject-378222.Capstone_Project.Academic_score'

# Defining the schema for the new table
schema = [
    bigquery.SchemaField('ID', 'INTEGER', mode='REQUIRED'),
    bigquery.SchemaField('Acd_score', 'FLOAT'),
  ]

try:
    # Try to get the existing table
    table = client.get_table(table_id)
    print(f'Table {table.table_id} already exists in BigQuery.')
except google.api_core.exceptions.NotFound:
    # If the table does not exist, create it
    table = bigquery.Table(table_id, schema=schema)
    table = client.create_table(table)
    print(f'Table {table.table_id} created in BigQuery.')


project_id = 'surveyproject-378222'

#Inserting Data into to Big-Query Base Table
client = bigquery.Client(project=project_id)
job_config = bigquery.LoadJobConfig()
job_config.source_format = bigquery.SourceFormat.CSV
job_config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE
job = client.load_table_from_dataframe(Academic_score, table_id, job_config=job_config)
job.result()

Table Academic_score created in BigQuery.


LoadJob<project=surveyproject-378222, location=northamerica-northeast2, id=1ce14dc6-20db-4f88-87e4-d06dba108000>

In [ ]:
#Reading data from BigQuery to view the push data
Academic_score= pandas_gbq.read_gbq(f'SELECT * FROM `surveyproject-378222.Capstone_Project.Academic_score`',project_id='surveyproject-378222', dialect='standard') 

Downloading: 100%|██████████|


In [ ]:
Academic_score.head(10)

,ID,Acd_score
0,347,2.0
1,320,2.0
2,21,2.0
3,171,2.0
4,73,2.0
5,342,2.0
6,155,2.0
7,299,2.0
8,303,2.0
9,17,2.0


## Creating Accommodation Score Table 

In [ ]:
Accommodation_feedback=pandas_gbq.read_gbq(f'SELECT * FROM `surveyproject-378222.Capstone_Project.Accommodation_feedback`',project_id='surveyproject-378222', dialect='standard')

Downloading: 100%|██████████|


In [ ]:
acf = Accommodation_feedback.copy()
label_map = {'\xa0Dissatisfied': 2, 'Neutral': 3, 'Highly Dissatisfied': 1, 'Satisfied': 4, 'Highly Satisfied': 5}
cols_to_map = ['Accm_finding_score', 'Accm_Quality_Score', 'Accm_affordibility_score','Commute_score','Needs_Availibility_Score']
acf[cols_to_map] = acf[cols_to_map].applymap(label_map.get)
weights = {'A': 0.20, 'B': 0.20, 'C': 0.20, 'D': 0.20, 'E': 0.20}
acf['Acc_score'] = ((acf['Accm_finding_score']*weights['A']) + (acf['Accm_Quality_Score']*weights['B']) + (acf['Accm_affordibility_score']*weights['C'])+ (acf['Commute_score']*weights['D']) + (acf['Needs_Availibility_Score']*weights['E']))
acf =acf[['ID','Acc_score']]
Accommodation_score = acf


In [ ]:
#Connecting Big-Query Project 
client = bigquery.Client(project='surveyproject-378222')
sc = """
DROP TABLE IF EXISTS `surveyproject-378222.Capstone_Project.Accommodation_score`;
"""
query_job = client.query(sc)
query_job.result()
table_id = 'surveyproject-378222.Capstone_Project.Accommodation_score'

# Defining the schema for the new table
schema = [
    bigquery.SchemaField('ID', 'INTEGER', mode='REQUIRED'),
    bigquery.SchemaField('Acc_score', 'FLOAT'),
  ]

try:
    # Try to get the existing table
    table = client.get_table(table_id)
    print(f'Table {table.table_id} already exists in BigQuery.')
except google.api_core.exceptions.NotFound:
    # If the table does not exist, create it
    table = bigquery.Table(table_id, schema=schema)
    table = client.create_table(table)
    print(f'Table {table.table_id} created in BigQuery.')


project_id = 'surveyproject-378222'

#Inserting Data into to Big-Query Base Table
client = bigquery.Client(project=project_id)
job_config = bigquery.LoadJobConfig()
job_config.source_format = bigquery.SourceFormat.CSV
job_config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE
job = client.load_table_from_dataframe(Accommodation_score, table_id, job_config=job_config)
job.result()

Table Accommodation_score created in BigQuery.


LoadJob<project=surveyproject-378222, location=northamerica-northeast2, id=56068a4d-54e9-43b9-b008-3e76e8799782>

In [ ]:
#Reading data from BigQuery to view the push data
Accommodation_score= pandas_gbq.read_gbq(f'SELECT * FROM `surveyproject-378222.Capstone_Project.Accommodation_score`',project_id='surveyproject-378222', dialect='standard') 


Downloading: 100%|██████████|


In [ ]:
Accommodation_score

,ID,Acc_score
0,250,2.0
1,142,2.0
2,253,2.0
3,312,2.0
4,26,2.0
...,...,...
348,178,3.6
349,67,3.6
350,110,3.6
351,242,3.6


# Sentimental Analysis

In [ ]:
# Reading table feedback_data from google BigQuery
Feedback_data = pandas_gbq.read_gbq(f'SELECT * FROM `surveyproject-378222.Capstone_Project.Feedback`',project_id='surveyproject-378222', dialect='standard') 

Downloading: 100%|██████████|


### Data Preprocessing 

#### Normalizing the data

In [ ]:
fdk = Feedback_data
def  clean_text(df, text_field, new_text_field_name):
    df[new_text_field_name] = df[text_field].str.lower()
    df[new_text_field_name] = df[new_text_field_name].str.replace(r"@[A-Za-z0-9]+", "")  # remove Twitter usernames
    df[new_text_field_name] = df[new_text_field_name].str.replace(r"[^0-9A-Za-z \t]", "")  # remove non-alphanumeric characters
    df[new_text_field_name] = df[new_text_field_name].str.replace(r"http\S+", "")  # remove URLs
    df[new_text_field_name] = df[new_text_field_name].str.replace(r"^rt", "")  # remove retweet indicator
    df[new_text_field_name] = df[new_text_field_name].str.strip()  # remove leading/trailing whitespace 
    # remove numbers
    df[new_text_field_name] = df[new_text_field_name].str.replace('\d+', '') 
    return df
data_clean = clean_text(fdk , 'Feedback', 'text_clean')
data_clean.head()

,ID,Feedback,text_clean
0,19,None,None
1,209,None,None
2,60,None,None
3,127,None,None
4,195,None,None


#### Stopwords

In [ ]:
stop_words = set(stopwords.words('english'))
data_clean['text_clean'] = data_clean['text_clean'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]) if x is not None else '')

#### Tokenization

In [ ]:
data_clean['text_tokens'] = data_clean['text_clean'].apply(lambda x: word_tokenize(x))

#### Stemming

In [ ]:
def word_stemmer(Feedback):
    stem_text = [PorterStemmer().stem(i) for i in Feedback]
    return stem_text
data_clean['text_tokens_stem'] = data_clean['text_tokens'].apply(lambda x: word_stemmer(x))

#### Lemmatization

In [ ]:
def word_lemmatizer(Feedback):
    lem_text = [WordNetLemmatizer().lemmatize(i) for i in Feedback]
    return lem_text
data_clean['text_tokens_lemma'] = data_clean['text_tokens'].apply(lambda x: word_lemmatizer(x))


### Sentiment Analysis



#### The sentiment scores for each unique identifier

In [ ]:
# Instantiate the SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()

# Define a function to apply the sentiment analyzer to each row of text
def get_sentiment_score(text):
    text_str = ' '.join(text)
    return sia.polarity_scores(text_str)['compound']


# Drop the rows containing None values in the 'text_tokens_lemma' column
fdk = fdk.dropna(subset=['text_tokens_lemma'])

# Apply the function to the 'text_tokens_lemma' column and create a new column with the sentiment scores
fdk['sentiment_score'] = fdk['text_tokens_lemma'].apply(get_sentiment_score)

# Group the DataFrame by the unique identifier and calculate the mean sentiment score for each group
grouped = fdk.groupby('ID')['sentiment_score'].mean()

In [ ]:
# Define a function to label each score as positive, negative, or neutral
def get_sentiment_label(score):
    if score > 0:
        return 'Positive'
    elif score < 0:
        return 'Negative'
    else:
        return 'Neutral'

# Apply the function to the sentiment_score column to create three new columns
fdk['Positive Score'] = fdk['sentiment_score'].apply(lambda x: max(0, x))
fdk['Negative Score'] = fdk['sentiment_score'].apply(lambda x: min(0, x))
fdk['Neutral Score'] = fdk['sentiment_score'].apply(lambda x: 0 if x == 0 else None)

### Sentiment Analysis with Word Counts for Each Sentiment Label

In [ ]:
# Define a function to calculate the sentiment label for a given word
def get_sentiment_label(word):
    score = TextBlob(word).sentiment.polarity
    return 'Positive' if score > 0 else ('Negative' if score < 0 else 'Neutral')

# Apply the function to each word in the text_tokens_lemma column
sentiment_labels = []
for text in fdk['text_tokens_lemma']:
    sentence = ' '.join(text)  # Join the list of words back into a sentence
    for word in text:
        sentiment_labels.append((get_sentiment_label(word), word))

# Create a DataFrame with the sentiment label, the word, and its count
sentiment_counts = pd.DataFrame(sentiment_labels, columns=['sentiment', 'word']) \
    .groupby(['sentiment', 'word']) \
    .size() \
    .reset_index(name='count')


sentiment_counts

,sentiment,word,count
0,Negative,active,1
1,Negative,average,1
2,Negative,bad,3
3,Negative,center,1
4,Negative,confusing,1
...,...,...,...
610,Positive,super,1
611,Positive,supportive,1
612,Positive,thanks,1
613,Positive,whole,1


In [ ]:
sentiment_score = fdk[['ID','sentiment_score']]
sentiment_score.head(3)

,ID,sentiment_score
0,19,0.0
1,209,0.0
2,60,0.0


### Creating schema for above tables

In [ ]:
#Connecting Big-Query Project 
client = bigquery.Client(project='surveyproject-378222')
sc = """
DROP TABLE IF EXISTS `surveyproject-378222.Capstone_Project.Sentiment_counts`;
"""
query_job = client.query(sc)
query_job.result()
table_id = 'surveyproject-378222.Capstone_Project.Sentiment_counts'

# Defining the schema for the new table
schema = [
    bigquery.SchemaField('sentiment', 'STRING'),
    bigquery.SchemaField('word', 'STRING'),
    bigquery.SchemaField('count', 'INTEGER'),
    ]

try:
    # Try to get the existing table
    table = client.get_table(table_id)
    print(f'Table {table.table_id} already exists in BigQuery.')
except google.api_core.exceptions.NotFound:
    # If the table does not exist, create it
    table = bigquery.Table(table_id, schema=schema)
    table = client.create_table(table)
    print(f'Table {table.table_id} created in BigQuery.')


project_id = 'surveyproject-378222'

#Inserting Data into to Big-Query Base Table
client = bigquery.Client(project=project_id)
job_config = bigquery.LoadJobConfig()
job_config.source_format = bigquery.SourceFormat.CSV
job_config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE
job = client.load_table_from_dataframe(sentiment_counts, table_id, job_config=job_config)
job.result()

Table Sentiment_counts created in BigQuery.


LoadJob<project=surveyproject-378222, location=northamerica-northeast2, id=6439672d-6cd4-4386-8f90-53572e6c6719>

In [ ]:
#Connecting Big-Query Project 
client = bigquery.Client(project='surveyproject-378222')
sc = """
DROP TABLE IF EXISTS `surveyproject-378222.Capstone_Project.Sentiment_score`;
"""
query_job = client.query(sc)
query_job.result()
table_id = 'surveyproject-378222.Capstone_Project.Sentiment_score'

# Defining the schema for the new table
schema = [
    bigquery.SchemaField('ID', 'INTEGER',mode='REQUIRED'),
    bigquery.SchemaField('sentiment_score', 'FLOAT')
    ]
try:
    # Try to get the existing table
    table = client.get_table(table_id)
    print(f'Table {table.table_id} already exists in BigQuery.')
except google.api_core.exceptions.NotFound:
    # If the table does not exist, create it
    table = bigquery.Table(table_id, schema=schema)
    table = client.create_table(table)
    print(f'Table {table.table_id} created in BigQuery.')


project_id = 'surveyproject-378222'

#Inserting Data into to Big-Query Base Table
client = bigquery.Client(project=project_id)
job_config = bigquery.LoadJobConfig()
job_config.source_format = bigquery.SourceFormat.CSV
job_config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE
job = client.load_table_from_dataframe(sentiment_score, table_id, job_config=job_config)
job.result()

Table Sentiment_score created in BigQuery.


LoadJob<project=surveyproject-378222, location=northamerica-northeast2, id=80a72114-0543-493c-95b3-3abfa750cefc>

In [ ]:
Sentiment_score = pandas_gbq.read_gbq(f'SELECT * FROM `surveyproject-378222.Capstone_Project.Sentiment_score`',project_id='surveyproject-378222', dialect='standard') 
Sentiment_count = pandas_gbq.read_gbq(f'SELECT * FROM `surveyproject-378222.Capstone_Project.Sentiment_counts`',project_id='surveyproject-378222', dialect='standard') 

Downloading: 100%|██████████|
Downloading: 100%|██████████|


In [ ]:
Sentiment_count.head(3)

,sentiment,word,count
0,Neutral,accent,1
1,Neutral,access,1
2,Neutral,accommodation,1


In [ ]:
Sentiment_score.head(3)

,ID,sentiment_score
0,19,0.0
1,209,0.0
2,60,0.0


# END of this File